# 4일차 4교시 조인

### 목차
* 1. 조인 유형
* 2. Inner Join
* 3. Outer Join
* 4. Semi Join
* 5. Anti Join
* 6. Natural Join
* 7. Cross Join
* 8. 조인 유의사항
* 9. 조인 문제점

- 선택한 조인 타입에 따라 조인의 성능에 영향을 미침. 
- 조인할 때 익스큐터에서 실행 파일 간의 데이터 셔플링을 필요로 하기 때문에 다른 조인과 조인순서를 고려해야함

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Data Engineer Intermediate Day4") \
    .config("spark.dataengineer.intermediate.day4", "tutorial-4") \
    .getOrCreate()

### 1. 조인 유형

+ 스파크의 조인 타입
    + inner join, outer join, left outer join, right outer join
    + left semi join: 왼쪽 데이터셋의 키가 오른쪽 데이터셋에 있는 경우에는 키가 일치하는 왼쪽 데이터셋만 유지 
    + left anti join: 왼쪽 데이터셋의 키가 오른쪽 데이터셋에 없는 경우에는 키가 일치하지 않는 왼쪽 데이터셋만 유지
    + natural join: 두 데이터셋에서 동일한 이름을 가진 컬럼을 암시적으로 결합하는 조인을 수행
    + cross join: 왼쪽 데이터셋의 모든 로우와 오른쪽 데이터 셋의 모든 로우를 조합

![join](image/join.png)

#### 예제

In [3]:
""" 예제에서 사용할 데이터 셋 """
person = spark.createDataFrame([
    (0, "Bill Chambers", 0, [100]),
    (1, "Matei Zaharia", 1, [500, 250, 100]),
    (2, "Michael Arnbrust", 1, [250, 100])
    ]) \
    .toDF("id", "name", "graduate_program", "spark_status")

graduateProgram = spark.createDataFrame([
    (0, "Masters", "School of Information", "UC Berkeley"),
    (2, "Masters", "EECS", "UC Berkeley"),
    (1, "Ph. D", "EECS", "UC Berkeley")
    ]) \
    .toDF("id", "degree", "department", "school")

sparkStatus = spark.createDataFrame([
    (500, "Vice President"),
    (250, "PMC Member"),
    (100, "Contributor")
    ])\
    .toDF("id", "status")

In [4]:
person.createOrReplaceTempView("person")
graduateProgram.createOrReplaceTempView("graduateProgram")
sparkStatus.createOrReplaceTempView("sparkStatus")

In [5]:
print("person")
person.printSchema()
person.show()

print("graduateProgram")
graduateProgram.printSchema()
graduateProgram.show()

print("sparkStatus")
sparkStatus.printSchema()
sparkStatus.show()

person
root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- graduate_program: long (nullable = true)
 |-- spark_status: array (nullable = true)
 |    |-- element: long (containsNull = true)

+---+----------------+----------------+---------------+
| id|            name|graduate_program|   spark_status|
+---+----------------+----------------+---------------+
|  0|   Bill Chambers|               0|          [100]|
|  1|   Matei Zaharia|               1|[500, 250, 100]|
|  2|Michael Arnbrust|               1|     [250, 100]|
+---+----------------+----------------+---------------+

graduateProgram
root
 |-- id: long (nullable = true)
 |-- degree: string (nullable = true)
 |-- department: string (nullable = true)
 |-- school: string (nullable = true)

+---+-------+--------------------+-----------+
| id| degree|          department|     school|
+---+-------+--------------------+-----------+
|  0|Masters|School of Informa...|UC Berkeley|
|  2|Masters|                EE

### 2. Left Inner Join
- join type 을 명시하지 않았을 때 기본적으로 내부조인을 수행함
- 왼쪽 테이블과 오른쪽 테이블에서 동일한 칼럼을 가져야함. 
+ 참으로 평가되는 로우만 결합
+ 세 번째 파라미터로 조인 타입을 명확하게 지정할 수 있음
- 성능 팁 : 두 테이블의 키가 중복되거나 여러 복사본으로 있다면 성능이 저하됨. 조인이 여러 키를 최소화하기 위해 일종의 카테시안 조인으로 변환되어 버림.

In [6]:
joinExpression = person["graduate_program"] == graduateProgram["id"]
person.join(graduateProgram, joinExpression).show()

+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
| id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph. D|                EECS|UC Berkeley|
|  2|Michael Arnbrust|               1|     [250, 100]|  1|  Ph. D|                EECS|UC Berkeley|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+



In [7]:
joinExpression = person["graduate_program"] == graduateProgram["id"]
person.join(graduateProgram, joinExpression, "inner").show()

+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
| id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph. D|                EECS|UC Berkeley|
|  2|Michael Arnbrust|               1|     [250, 100]|  1|  Ph. D|                EECS|UC Berkeley|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+



### 3. Outer Join
- 왼쪽과 오른쪽의 모든 로우를 제공. 
+ 왼쪽이나 오른쪽 DataFrame에 일치하는 로우가 없다면 해당 위치에 null을 삽입
- 성능 팁 : 공통 로우가 거의 없는 테이블에서 사용하면 결과값이 매우 커지고 성능 저하

In [8]:
joinExpression = person["graduate_program"] == graduateProgram["id"]
person.join(graduateProgram, joinExpression, "outer").show() # "outer"는 외부 조인

+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
|   0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|   1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph. D|                EECS|UC Berkeley|
|   2|Michael Arnbrust|               1|     [250, 100]|  1|  Ph. D|                EECS|UC Berkeley|
|null|            null|            null|           null|  2|Masters|                EECS|UC Berkeley|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+



#### 3-1. Left Outer Join
- 왼쪽의 모든 로우와 오른쪽의 공통 로우(inner join)를 제공. 오른쪽에 일치하는 로우가 없으면 null을 채움

In [9]:
person.join(graduateProgram, joinExpression, "left_outer").show() # "outer"는 외부 조인

+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
| id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph. D|                EECS|UC Berkeley|
|  2|Michael Arnbrust|               1|     [250, 100]|  1|  Ph. D|                EECS|UC Berkeley|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+



#### 3-2. Right Outer Join
- 식의 순서만 바꾸면 왼쪽 외부 조인과 같은 역할을 함. 

In [10]:
person.join(graduateProgram, joinExpression, "right_outer").show() # "outer"는 외부 조인

+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
|   0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|   1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph. D|                EECS|UC Berkeley|
|   2|Michael Arnbrust|               1|     [250, 100]|  1|  Ph. D|                EECS|UC Berkeley|
|null|            null|            null|           null|  2|Masters|                EECS|UC Berkeley|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+



In [11]:
graduateProgram.join(person, joinExpression, "left_outer").show()

+---+-------+--------------------+-----------+----+----------------+----------------+---------------+
| id| degree|          department|     school|  id|            name|graduate_program|   spark_status|
+---+-------+--------------------+-----------+----+----------------+----------------+---------------+
|  0|Masters|School of Informa...|UC Berkeley|   0|   Bill Chambers|               0|          [100]|
|  1|  Ph. D|                EECS|UC Berkeley|   1|   Matei Zaharia|               1|[500, 250, 100]|
|  1|  Ph. D|                EECS|UC Berkeley|   2|Michael Arnbrust|               1|     [250, 100]|
|  2|Masters|                EECS|UC Berkeley|null|            null|            null|           null|
+---+-------+--------------------+-----------+----+----------------+----------------+---------------+



### 4. Left Semi Join
- 오른쪽에 존재하는 것을 기반으로 왼쪽 로우만 제공
+ 값이 존재하는 지 확인 용도, 값이 있다면 왼쪽 DataFrame에 중복 키가 있더라도 해당 로우는 결과에 포함
+ 기존 조인 기능과는 달리 DataFrame의 필터 기능과 유사
- 성능: 하나의 테이블만 확실히 고려되고, 다른 테이블은 조인 조건만 확인하기 때문에 성능이 매우 좋음

In [12]:
# 오른쪽 person 에 있는 graduate_program = 0, 1 밖에 없으므로 그 기준으로 graduatePrograme 을 출력합니다
joinType = 'left_semi'
graduateProgram.join(person, joinExpression, joinType).show(5)

+---+-------+--------------------+-----------+
| id| degree|          department|     school|
+---+-------+--------------------+-----------+
|  0|Masters|School of Informa...|UC Berkeley|
|  1|  Ph. D|                EECS|UC Berkeley|
+---+-------+--------------------+-----------+



In [14]:
""" 중복 키가 존재하더라도 그대로 출력됩니다 """
duplicatedGraduateProgram = graduateProgram.union(spark.createDataFrame([
    (0, 'Masters', "Duplicated Row", "Duplicated School")
]))
duplicatedGraduateProgram.createOrReplaceTempView("duplicatedGraduateProgram")
duplicatedGraduateProgram.join(person, joinExpression, joinType).show(5)

+---+-------+--------------------+-----------------+
| id| degree|          department|           school|
+---+-------+--------------------+-----------------+
|  0|Masters|School of Informa...|      UC Berkeley|
|  0|Masters|      Duplicated Row|Duplicated School|
|  1|  Ph. D|                EECS|      UC Berkeley|
+---+-------+--------------------+-----------------+



### 5. Left Anti Join
+ 왼쪽 세미 조인의 반대 개념, 즉 오른쪽 DataFrame의 어떤 값도 포함하지 않음
+ SQL의 NOT IN과 같은 스타일의 필터

In [15]:
joinType = 'left_anti'
graduateProgram.join(person, joinExpression, joinType).show()

+---+-------+----------+-----------+
| id| degree|department|     school|
+---+-------+----------+-----------+
|  2|Masters|      EECS|UC Berkeley|
+---+-------+----------+-----------+



### 6. Natural Join
+ 조인하려는 컬럼을 암시적으로 추정
+ 암시적인 처리는 언제나 위험하므로 비추천
+ Python join 함수는 이 기능을 지원하지 않음


```
-- SQL
SELECT * FROM graduateProgram NATURAL JOIN person
```

In [17]:
# 지정된 필드의 값이 일치하는 경우 해당 필드를 기준으로 조인
spark.sql("SELECT * FROM graduateProgram NATURAL JOIN person").show()

+---+-------+--------------------+-----------+----------------+----------------+---------------+
| id| degree|          department|     school|            name|graduate_program|   spark_status|
+---+-------+--------------------+-----------+----------------+----------------+---------------+
|  0|Masters|School of Informa...|UC Berkeley|   Bill Chambers|               0|          [100]|
|  1|  Ph. D|                EECS|UC Berkeley|   Matei Zaharia|               1|[500, 250, 100]|
|  2|Masters|                EECS|UC Berkeley|Michael Arnbrust|               1|     [250, 100]|
+---+-------+--------------------+-----------+----------------+----------------+---------------+



### 7. Cross Join - Cartesian Join
+ 교차 조인은 조건절을 기술하지 않은 내부 조인을 의미
+ 왼쪽의 모든 로우를 오른쪽의 모든 로우와 결합함(결과의 로우 수 = 왼쪽 로우 수 * 오른쪽 로우 수)
- 성능 : 큰 데이터에서 사용할 경우 out-of-memory exception 발생. 가장 좋지 않은 성능을 가진 조인. 주의해서 사용하며 특정 사례에서만 사용해야함.

In [18]:
""" 크로스 조인이지만 조건을 설정해야 하며, 조건에 부합된 결과를 출력하여 inner조인과 동일."""
joinType = "cross"
graduateProgram.join(person, on=joinExpression, how=joinType).show()

+---+-------+--------------------+-----------+---+----------------+----------------+---------------+
| id| degree|          department|     school| id|            name|graduate_program|   spark_status|
+---+-------+--------------------+-----------+---+----------------+----------------+---------------+
|  0|Masters|School of Informa...|UC Berkeley|  0|   Bill Chambers|               0|          [100]|
|  1|  Ph. D|                EECS|UC Berkeley|  1|   Matei Zaharia|               1|[500, 250, 100]|
|  1|  Ph. D|                EECS|UC Berkeley|  2|Michael Arnbrust|               1|     [250, 100]|
+---+-------+--------------------+-----------+---+----------------+----------------+---------------+



In [19]:
person.crossJoin(graduateProgram).show() #spark 2.1 이후

+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
| id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|  0|   Bill Chambers|               0|          [100]|  2|Masters|                EECS|UC Berkeley|
|  0|   Bill Chambers|               0|          [100]|  1|  Ph. D|                EECS|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  0|Masters|School of Informa...|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  2|Masters|                EECS|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph. D|                EECS|UC Berkeley|
|  2|Michael Arnbrust|               1|     [250, 100]|  0|Masters|School of Informa...|UC 

### 8. 조인 시 유의 사항
- theta 조건을 충족하면 join되는 [Theta Join - Tutorials point](https://www.tutorialspoint.com/dbms/database_joins.htm): key끼리 매치되도록 버켓을 만듦
- 한 테이블의 한 열과 두번째 테이블의 여러 열과 매핑되는 One to Many Join : 한 열의 크기가 작으므로 일반적으로 메모리를 신경 쓰지 않아도 됨. [parquet](http://engineering.vcnc.co.kr/2018/05/parquet-and-spark/)을 사용하면 문제가 안됨.  
- 하나의 테이블이 자신을 조인하는 self join : 데이터의 크기가 클 경우 out-of-memory 발생할 수 있음

----

### 9. 조인 사용 시 문제점

#### 9.1 복합 데이터 타입의 조인
+ 불리언을 반환하는 모든 표현식은 조인 표현식으로 간주할 수 있음

In [20]:
from pyspark.sql.functions import expr

person.withColumnRenamed("id", "personId") \
    .join(sparkStatus, expr("array_contains(spark_status, id)")) \
    .show()

+--------+----------------+----------------+---------------+---+--------------+
|personId|            name|graduate_program|   spark_status| id|        status|
+--------+----------------+----------------+---------------+---+--------------+
|       0|   Bill Chambers|               0|          [100]|100|   Contributor|
|       1|   Matei Zaharia|               1|[500, 250, 100]|500|Vice President|
|       1|   Matei Zaharia|               1|[500, 250, 100]|250|    PMC Member|
|       1|   Matei Zaharia|               1|[500, 250, 100]|100|   Contributor|
|       2|Michael Arnbrust|               1|     [250, 100]|250|    PMC Member|
|       2|Michael Arnbrust|               1|     [250, 100]|100|   Contributor|
+--------+----------------+----------------+---------------+---+--------------+



#### 9.2 중복 컬럼명 처리


In [21]:
""" 잘못된 데이터셋(컬럼명 중복) """
gradProgramDupe = graduateProgram.withColumnRenamed("id", "graduate_program")
joinExpr = gradProgramDupe["graduate_program"] == person["graduate_program"]

In [22]:
gradProgramDupe.join(person, joinExpr, "inner").show() # 조인을 수행했음에도 두 개의 graduate_program 컬럼이 존재

+----------------+-------+--------------------+-----------+---+----------------+----------------+---------------+
|graduate_program| degree|          department|     school| id|            name|graduate_program|   spark_status|
+----------------+-------+--------------------+-----------+---+----------------+----------------+---------------+
|               0|Masters|School of Informa...|UC Berkeley|  0|   Bill Chambers|               0|          [100]|
|               1|  Ph. D|                EECS|UC Berkeley|  1|   Matei Zaharia|               1|[500, 250, 100]|
|               1|  Ph. D|                EECS|UC Berkeley|  2|Michael Arnbrust|               1|     [250, 100]|
+----------------+-------+--------------------+-----------+---+----------------+----------------+---------------+



In [23]:
""" 중복 컬럼 조회 시 오류 발생 """
# gradProgramDupe.join(person, joinExpr, "inner").select("graduate_program").show()

' 중복 컬럼 조회 시 오류 발생 '

In [24]:
""" 해결 방법 1: 다른 조인 표현식 사용 """
# 중복된 두 컬럼 중 하나가 자동 제거 됨
person.join(gradProgramDupe, "graduate_program").select("graduate_program").show()

+----------------+
|graduate_program|
+----------------+
|               0|
|               1|
|               1|
+----------------+



In [25]:
""" 해결 방법 2: 조인 후 컬럼 제거 """
gradProgramDupe.join(person, joinExpr) \
    .drop(gradProgramDupe["graduate_program"]) \
    .select("graduate_program") \
    .show() 

+----------------+
|graduate_program|
+----------------+
|               0|
|               1|
|               1|
+----------------+



In [26]:
""" 해결 방법 3: 조인 전 컬럼명 변경 (가장 확실한 방법) """
fixed_gradProgram = gradProgramDupe.withColumnRenamed("graduate_program", "grad_id")
fixed_gradProgram.join(person, fixed_gradProgram["grad_id"] == person["graduate_program"]).show()

+-------+-------+--------------------+-----------+---+----------------+----------------+---------------+
|grad_id| degree|          department|     school| id|            name|graduate_program|   spark_status|
+-------+-------+--------------------+-----------+---+----------------+----------------+---------------+
|      0|Masters|School of Informa...|UC Berkeley|  0|   Bill Chambers|               0|          [100]|
|      1|  Ph. D|                EECS|UC Berkeley|  1|   Matei Zaharia|               1|[500, 250, 100]|
|      1|  Ph. D|                EECS|UC Berkeley|  2|Michael Arnbrust|               1|     [250, 100]|
+-------+-------+--------------------+-----------+---+----------------+----------------+---------------+



----

### 10. 스파크의 조인 수행 방식

### Broadcast Hash Join : **SmallSize** join **AnySize**
- 큰 데이터셋과 이보다 작은 데이터 셋 간의 조인은 큰 데이터셋의 파티션이 있는 모든 익스큐터에 작은 데이터셋이 브로드캐스트되어 수행
- 기본값: spark.sql.autoBroadcastJoinThreshold = 10mb 
  - spark.sql.autoBroadcastJoinThreshold 는 조인을 수행 할 때 모든 작업자 노드에 브로드캐스트되는 테이블의 최대 크기를 구성
  - 참고
    - [Spark Doc - performance Tuning](https://spark.apache.org/docs/latest/sql-performance-tuning.html)
    - [Does spark.sql.autoBroadcastJoinThreshold work for joins using Dataset's join operator?](https://stackoverflow.com/questions/43984068/does-spark-sql-autobroadcastjointhreshold-work-for-joins-using-datasets-join-op)
- 수행
  - 작은 테이블은 드라이버에 모였다가 다시 모든 노드에 복사
  - 작은 테이블은 메모리에 올라가고
  - 큰 테이블은 스트림을 통해서 조인 수행
  - 브로드캐스트 힌트 `person.join(broadcast(grudateProgram),Seq(“id”))`
    - `autoBroadcastJoinThreshold`에 관계없이 힌트가 있는 조인 측이 브로드캐스트됨. 
    - 조인의 양쪽에 브로드캐스트 힌트가 있으면 실제 크기가 더 작은 쪽이 브로드 캐스트됨. 
    - 힌트가 없고 테이블의 실제 물리적 추정값이 autoBroadcastJoinThreshold 보다 작으면, 해당 테이블은 모든 실행기 노드로 브로드캐스트됨.
- 성능 
  - 한 쪽의 데이터가 하나의 machine 에 fit-in-meory 될 정도로 작으면 성능 좋음.
    - table broadcast는 네트워크를 많이 사용하므로, 브로드캐스트 된 테이블이 크면 때때로 out-of-memory나 성능저하가 발생할 수 있음
  - 셔플링이 없기 때문에, 브로드캐스트 되는 쪽이 데이터가 작으면 다른 알고리즘보다 빠름. 
- 브로드캐스트 지원
  - Full outer join 은 지원하지 않음. 
  - letf-outer join 에서는 오른쪽 테이블만 브로드캐스트, right-outer join 에서는 왼쪽 테이블만 브로드캐스트 가능.

### Shuffle hash join : **MiddleSize** join **LargeSize**
- 두 테이블 모두 shuffle 을 통해 노드에 분산되고
- 비교적 작은 테이블이 메모리 버퍼에 올라가고
- 큰 테이블은 스트림을 통해서 조인 수행
- 파티션이 전체 익스큐터로 분배
- 셔플은 비용이 많이 듦. 파티션과 셔플 배포가 최적으로 수행되는지 확인하기 위해 로직을 분석하는 것이 중요.
  - 큰 데이터는 join에 필요한 부분만 filtering 을 하거나
  - repartioning 을 고려해야함

- Map Reduce Fundamentals 유사
  - Map - 두 개의 서로 다른 Data frames/table
    - Output key를 join 조건에서 필드로 사용
    - Shuflle - output key로 두 데이터 세트를 섞음
  - Reduce - join 결과

![shuffle hash join](https://i.pinimg.com/originals/48/41/81/4841810dd7ad50397d566b8c9beb7875.jpg)

#### 성능을 최적화하려면
- join할 키가 균등하게 dirstribute되어있거나, 
- parallelism위한 적절한 수의 키가 있을 때
  
#### 성능이 나쁜 경우 - 고르지 않은 sharding 및 제한된 parallelism
- data skewness 처럼 하나의 단일 파티션이 다른 파티션에 비해 너무 많은 데이터를 가지고 있을 때
- 각 스테이트에서 50개 키만 셔플할 수 있음 -> 스파크 클러스터가 크면 고른 sharding과 parallelism 으로 해결 못함

![problem of shuffle hash join](https://image.slidesharecdn.com/optimizingsparksqljoins-170209164631/95/optimizing-apache-spark-sql-joins-11-638.jpg?cb=1486658917)

### Sort merge join : **LargeSize** join **LargeSize**
- 일치하는 조인키를 sort할 수 있고, 브로드캐스트 조인, 셔플 해시 조인에 적합하지 않은 경우 사용
- shuffle hash join 과 비교했을 때, 클러스터에서 데이터 이동(shuffling)을 최소화함
- 수행
  - 두 테이블 모두 셔플 및 정렬이 발생하고
  - 그나마 작은 쪽이 버퍼를 하고 큰 쪽이 스트리밍으로 조인을 수행한다
  - partition 은 join 작업 전에 조인키 정렬
- 참고. [SortMergeJoinExec Binary Physical Operator for Sort Merge Join](https://jaceklaskowski.gitbooks.io/mastering-spark-sql/spark-sql-SparkPlan-SortMergeJoinExec.html)

### BroadcastNestedLoopJoin
- 적용 : 조인 키가 지정되어 있지 않고 브로드 캐스트 힌트가 있거나 조인의 한쪽이 브로드캐스트 될 수 있고, spark.sql.autoBroadcastJoinThreshold보다 작은 경우
- 브로드캐스트 된 데이터 세트가 크면 매우 느릴 수 있으며 OutOfMemoryExceptions을 일으킬 수 있음

---

### 테이블 크기에 따른 조인 동작방식

#### 큰 테이블과 큰 테이블 조인
+ 전체 노드 간 통신이 발생하는 셔플 조인이 발생됨

#### 큰 테이블과 작은 테이블 조인
+ 작은 DataFrame을 클러스터 전체 워커에 복제한 후 통신없이 진행
+ 모든 단일 노드에서 개별적으로 조인이 수행되므로 CPU가 가장 큰 병목 구간이 됨
+ broadcast 함수(힌트)를 통해 브로드캐스트 조인을 설정할 수 있으나 강제할 수는 없음(옵티마이저가 무시 가능)

In [27]:
from pyspark.sql.functions import broadcast
person.join(broadcast(gradProgramDupe), "graduate_program").select("graduate_program").show()

+----------------+
|graduate_program|
+----------------+
|               0|
|               1|
|               1|
+----------------+



#### 작은 테이블과 작은 테이블 조인
+ 스파크가 결정하도록 내버려두는 것이 제일 좋은 선택

### Check
- 브로드캐스트조인을 가능한 한 사용하고 조인 전에 관련없는 행을 조인 키로 필터링하여 불필요한 데이터 셔플링을 피하자
  - 필요한 경우 spark.sql.autoBroadcastJoinThreshold를 적절히 조정
- sort-merge join 이 default이고, 대부분의 시나리오에서 잘 수행됨. 
  - Shuffle Hash 조인이 Sort-Merge 조인보다 낫다고 확신이 있으면,Sort-Merge join을 비활성화해서 shuffle hash join이 수행되도록 함. 
    - builde size 가 stream size보다 작으면 Shuffle Hash 조인이 나음
- unique한 조인키가 없거나 조인키가 없는 조인은 수행비용이 비싸므로 최대한 피해야함

---

## 설명 이해 위한 배경 지식

### 파티셔닝 partioning 
- 어떤 데이터를 어디에 저장할 것인지 제어할 수 있는 기능
- RDD는 데이터 파티션으로 구성되고 모든 연산은 RDD의 데이터 파티션에서 수행됨. 
- 파티션 개수는 RDD 트랜스포메이션 실행할 태스크 수에 직접적인 영향을 줌
  - 파티션 개수 너무 적으면 -> 많은 데이터에서 아주 일부의 CPU/코어만 사용 -> 성능 저하, 클러스터 제대로 활용 못함
  - 파티션 개수 너무 많으면 -> 실제 필요한 것보다 많은 자원을 사용 -> 멀티테넌트 환경에서는 자원 부족 현상 발생
- Partioner 에 의해 RDD 파티셔닝이 실행된. 파티셔너는 파티션 인덱스를 RDD 엘리먼트에 할당. 

### Shuffling
- 파티셔너가 어떤 파티션을 사용하든 많은 연산이 RDD의 파티션 전체에 걸쳐 데이터 리파티셔닝Repartioning 이 발생함
  - 새로운 파티션이 생성되거나 파티션이 축소, 병합될 수 있음. 
- 리파티셔닝에 필요한 모든 데이터 이동을 **셔플링 Shuffling**이라고 함. 
  - **Shuffling 을 할 때, Disk I/O + Network I/O 과도하게 발생**
  - 셔플링은 계산을 동일 익스큐터의 메모리에서 더 이상 진행하지 않고 익스큐터 간에 데이터를 교환함 -> 많은 성능 지연을 초래할 수 있음 
  - 스파크 잡의 실행 프로세스를 결정, 잡이 스테이지로 분할되는 부분에 영향을 미침
  - 셔플링이 많을 수록 스파크 잡이 실행될 때 더 많은 스테이지가 발생하기 때문에 성능에 영향을 미침
- 리파티셔닝을 유발하는 연산은 **조인**, 리듀스, 그룹핑, 집계 연산이 있음

### 버켓팅 Bucketing
- Bucketing = pre-(shffle + sort) inputs on join Keys
- 각 파일에 저장된 데이터를 제어할 수 있는 또 다른 파일 조직화 기법
- 동일한 버킷 ID를 가진 데이터가 하나의 물리적 파티션에 모두 모여 있기 때문에 데이터를 읽을 때 셔플을 피할 수 있음
- **데이터가 이후 사용 방식에 맞춰 사전에 파티셔닝되므로 조인이나 집계할 때 발생하는 고비용의 셔플을 피할 수 있음.**
- 같은 키로 계속 조인이 발생하는 경우, 일별 누적으로 쌓여가는 테이블 을 버킷팅을 하면 효과를 볼 수 있음

### Broadcast variable
- 브로드캐스트 변수는 모든 익스큐터에서 사용할 수 있는 공유 변수 shared variable
- 드라이버에서 한 번 생성되면 익스큐터에서만 읽을 수 있음. 
- 전체 데이터셋이 스파크 클러스터에서 전파될 수 있어서 익스큐터에서는 브로드캐스트 변수의 데이터에 접근할 수 있음
- **익스큐터 내부에서 실행되는 모든 태스크는 모두 브로드캐스트 변수에 접근할 수 있음**

---

## Reference
- [Spark SQL, DataFrames and Datasets Guide](https://spark.apache.org/docs/latest/sql-programming-guide.html)
- [Spark summit 2017 - Hive Bucketing in Apache Spark with Tejas Patil](https://youtu.be/6BD-Vv-ViBw?t=30) / [slide](https://www.slideshare.net/databricks/hive-bucketing-in-apache-spark-with-tejas-patil) / [한글 요약본](https://www.notion.so/Hive-Bucketing-in-Apache-Spark-Tejas-Patil-9374879e0ca744cc8e7047e82cf5fdfa)
- [Spark summit 2017 - Optimizing Apache Spark SQL Joins: Spark Summit East talk by Vida Ha](https://www.youtube.com/watch?v=fp53QhSfQcI) / [slide](https://www.slideshare.net/databricks/optimizing-apache-spark-sql-joins)
- [Everyday I'm Shuffling - Tips for Writing Better Apache Spark Programs](https://www.youtube.com/watch?v=Wg2boMqLjCg)
- [Spark Memory Management by 0x0fff](https://0x0fff.com/spark-memory-management/)
- [Apache Spark에서 컬럼 기반 저장 포맷 Parquet(파케이) 제대로 활용하기](http://engineering.vcnc.co.kr/2018/05/parquet-and-spark/)
- [Understanding Database Sharding](https://www.digitalocean.com/community/tutorials/understanding-database-sharding)
- 도서 - 레자울 카림, 스리다 알라(2019). 빅데이터 분석을 위한 스칼라와 스파크(김용환 옮김). 에이콘 출판사 (원서 출판 2018)
- *그 외 레퍼런스 표기는 각 설명항목에 포함되어 있습니다*